In [92]:
import pandas as pd
pd.options.display.max_columns = None

In [93]:
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2019-02.parquet')
df_march = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2019-03.parquet')

In [94]:
df = pd.concat([df_feb, df_march])
df = df.drop_duplicates()

df = df.rename(columns={
            'VendorID':'vendor_id',
            'RatecodeID':'ratecode_id',
            'PULocationID':'pu_zone_id',
            'DOLocationID':'do_zone_id'})

In [96]:
# Outliers fast cleaning
df = df.drop(columns=['airport_fee'])

df = df.loc[df.trip_distance <= df.trip_distance.quantile(0.9)]
df = df.loc[df.trip_distance >= df.trip_distance.quantile(0.1)]

df = df.loc[df.fare_amount <= df.fare_amount.quantile(0.9)]
df = df.loc[df.fare_amount >= df.fare_amount.quantile(0.1)]


df = df.loc[df.ratecode_id!=99] #Unknown
df = df.loc[df.payment_type!=5] #Unknown

df = df.loc[df.passenger_count<=5]

In [98]:
dicc_ratecode = {1:"Standard", 2:"JFK", 3:"EWR", 4:"Nass/West", 5:"Negotiated", 6:"Group Ride", 99:"Unknown"}
df['ratecode'] = df.ratecode_id.map(dicc_ratecode)

df = df.drop(columns={'ratecode_id'})

dicc_payment = {1:"Credit Card", 2:"Cash", 3:"No charge", 4:"Dispute", 5:"Unknown", 6:"Voided trip"}
df['payment_type'] = df.payment_type.map(dicc_payment)

In [99]:
df['trip_distance_aux'] = df['trip_distance']

df.trip_distance_aux = df.trip_distance_aux.replace(0, 1)
df.trip_distance_aux = df.trip_distance_aux.fillna(1)

df['fare_per_mile'] = df.fare_amount / df.trip_distance_aux

df.loc[df['trip_distance'] == 0, 'fare_per_mile'] = 0

df = df.drop(columns=['trip_distance_aux'])

In [100]:
df['trip_time'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

df = df.loc[df.trip_time <= df.trip_time.quantile(0.9)]
df = df.loc[df.trip_time >= df.trip_time.quantile(0.1)]

df.trip_time = df.trip_time.dt.total_seconds()

df['trip_time_aux'] = df.trip_time
df.trip_time_aux = df.trip_time_aux.replace(0, 1)
df.trip_time_aux = df.trip_time_aux.fillna(1)

df['fare_per_minute'] = df.fare_amount / (df.trip_time_aux / 60)

df.loc[df['trip_time'] == 0, 'fare_per_minute'] = 0

df = df.drop(columns=['trip_time_aux'])

In [101]:
df_zone = pd.read_csv('https://raw.githubusercontent.com/soyHenry/DS-Proyecto_Grupal_TaxisNYC/main/taxi%2B_zone_lookup.csv')

df_zone = df_zone.rename(columns={
                    'LocationID':'id_zone',
                    'Borough':'borough',
                    'Zone':'zone'})


borough_data = {'id_borough': [1, 2, 3, 4, 5, 6, 7], 'borough':['Brooklyn', 'Bronx', 'Manhattan', 'Staten Island', 'Queens', 'EWR', 'Unknown']}
df_borough = pd.DataFrame(borough_data)

dic_zone_borough = {df_zone.id_zone[i] : df_zone.borough[i] for i in range (0,len(df_zone))}

dic_id_borough = {df_borough.borough[i] : df_borough.id_borough[i] for i in range (0, len(df_borough))}

df['pu_borough'] = df.pu_zone_id.map(dic_zone_borough)
df['do_borough'] = df.do_zone_id.map(dic_zone_borough)

df['id_borough'] = df.pu_borough.map(dic_id_borough)

df = df.drop(columns={'id_borough'})

In [115]:
df.passenger_count = df.passenger_count.astype(int)
df.to_csv('nyc_yellow_taxi_trips_2019_02_03.csv', index=False)